In [2]:
import pandas as pd
import datetime

In [3]:
# Reading raw data
raw_data = pd.read_csv("exportdata.csv")

# Get Current Month
# curr_month = datetime.datetime.strftime(datetime.datetime.now(), '%m')
curr_month = int(input("Enter Month Number : "))

# List of CIT bin
CIT_BIN = ['I-TBO-BI-CEN-OFSI-L2', 
           'I-TBO-BI-CEN-EAM-L2', 
           'I-TBO-BI-CEN-IGIS-L2', 
           'I-BI-TBO-CEN-CP-INFRATEL-CF', 
           'I-TBO-BA-CEN-TOC-OPS-L2']

Enter Month Number : 3


In [3]:
# Function to convert in date format
def conv_date(date):
    try:
        return datetime.datetime.strptime(date, '%b %d, %Y %I:%M %p')  # Jan 1, 2019 8:34 PM
    except:
        try:
            return datetime.datetime.strptime(date, '%Y-%m-%d %H:%M:%S')  # 2019-03-11 11:21:00
        except:
            return datetime.datetime.strptime(date, '%m/%d/%Y %H:%M')  # 1/1/2019 20:34

In [4]:
# Removing unwanted data
raw_data = raw_data.dropna(subset=['SolvedDate'], axis=0)
raw_data['SolvedDate'] = raw_data.apply(lambda x: conv_date(str(x['SolvedDate'])), axis=1)
raw_data['Solved_Month'] = raw_data['SolvedDate'].dt.month
raw_data = raw_data[raw_data['Solved_Month']==int(curr_month)]
raw_data = raw_data[raw_data['ticketType'] == 'INCIDENT']
raw_data = raw_data[raw_data['ResolverComp'] != 'AMS']
raw_data = raw_data[raw_data['ResolverGroup'].str.contains('L3')==False]


# Insert three blank columns
raw_data.insert(0, column='Ticket Category', value='')
raw_data.insert(1, column='Ticket Status', value='')
raw_data.insert(2, column='SLA Status', value='')

# Find position of 'InternalPriority' column
for index, col in enumerate(raw_data.columns):
    if col == 'InternalPriority':
        i = index

# Insert 'Resolution Time' column after 'InternalPriority' column
raw_data.insert(i+1, column='Resolution Time', value='')

In [5]:
# Function to get filter 'Ticket Category' 
def ticket_category(contactcode, affectedby, resolvergroup, statusboard, itdrescode):
    if contactcode == 'EVENTMANAGEMENT':
        return 'Event Management'
    elif contactcode == 'IBM INTERNAL':
        return 'IBM Internal'
    elif affectedby[0] == 'A':
        return 'IBM Internal'
    elif resolvergroup[0] == 'C':
        return 'Customer Owned'
    elif 'NONIBM' in resolvergroup:
        return 'Non IBM'
    elif 'RFC' in resolvergroup:
        return 'RFC Tickets'
    elif 'CHILD' in statusboard:
        return 'Child Ticket'
    elif 'DATA' in itdrescode:
        return 'Data Issue'
    elif 'KNOWLEDGE' in itdrescode:
        return 'Knowledge Issue'
    elif 'BIZ' in itdrescode:
        return 'BIZ Action'
    elif 'THIRD' in itdrescode:
        return 'Third Party'
    elif itdrescode == 'FIXCODE':
        return 'FIXCODE'
    elif 'DUPLICATE' in itdrescode:
        return 'DUPLICATE TICKET'
    else:
        return 'NA'

In [6]:
# Apply 'ticket_category' function to get Ticket Category
raw_data['Ticket Category'] = raw_data.apply(lambda x: ticket_category(str(x['contactcode']), str(x['Affectedby']), 
                                                                       str(x['ResolverGroup']), str(x['statusBoard']), 
                                                                       str(x['ITDRESCODE'])), axis=1)

In [7]:
# Function to update 'Ticket Status' 
def change_status(ticket_category):
    if ticket_category == "NA":
        return 'NA'
    else:
        return 'Exclusion'

In [8]:
# Applying function to update 'Ticket Status' and 'SLA Status' 
raw_data['Ticket Status'] = raw_data.apply(lambda x: change_status(x['Ticket Category']), axis=1)
raw_data['SLA Status'] = raw_data.apply(lambda x: change_status(x['Ticket Category']), axis=1)

In [9]:
# Function to update CIT and Non CIT
def cit(ticket_category, resolvergroup):
    if ticket_category == 'NA' and resolvergroup in CIT_BIN:
        return 'CIT'
    elif ticket_category == 'NA' and resolvergroup not in CIT_BIN:
        return 'Non CIT'
    else:
        return ticket_category

In [10]:
# Applying cit function to update CIT and Non CIT in 'Ticket Category' column
raw_data['Ticket Category'] = raw_data.apply(lambda x: cit(x['Ticket Category'], x['ResolverGroup']), axis=1)

In [11]:
# Function to update 'Ticket Status' 
def change_status2(ticket_category):
    if ticket_category == "CIT":
        return 'CIT'
    elif ticket_category == "Non CIT":
        return 'Non CIT'
    else:
        return 'Exclusion'

In [13]:
# Applying function to update 'Ticket Status' and 'SLA Status'
raw_data['Ticket Status'] = raw_data.apply(lambda x: change_status2(x['Ticket Category']), axis=1)
# raw_data['SLA Status'] = raw_data.apply(lambda x: change_status2(x['Ticket Category']), axis=1)

# Delete 'Solved_Month' column 
raw_data = raw_data.drop(labels=['Solved_Month'], axis=1)

In [14]:
raw_data.to_csv('final_rawdata.csv', index=False)
print("Final Data has been exported in csv format. Please goto homepage and download the output data. !!!")

Final Data has been exported in csv format. Please goto homepage and download the output data. !!!


In [18]:
# raw_data